In [1]:
import os
import glob
import pandas as pd 
import geopandas as gpd
import geoparquet as gpq
from utils import *
from congestion_metrics import *
import logging
# from simpledbf import Dbf5

In [2]:
SCENARIO_MAP = {'2018': 12,
                '2026': 16,
                '2031': 20,
                '2041': 22,
                '2051': 24 
                }

BASE_SCENARIO_MAP = SCENARIO_MAP.copy()
REVERSE_SCENARIO_MAP = {v: k for k, v in SCENARIO_MAP.items()}

FUTURE_SHP_DIR = '/Users/tszchun.chow/Library/CloudStorage/OneDrive-SharedLibraries-AucklandTransport/Congestion Charging - General/3. Policy & Strategy/Scheme analysis/Modelling outputs/011_Strategic_Case/Shape_Files/Shape_Files'

ROAD_LINKS_BASE_COLS = ['ID', 'INODE', 'JNODE', 'LENGTH', 'geometry']

In [4]:
## read nodes and links
def year(scen_code):
    if scen_code == 12:
        return "18"
    elif scen_code == 20:
        return "31"
    elif scen_code == 22:
        return "41"
    elif scen_code == 24:
        return "51"
    else:
        return str(YEAR)


def read_nodes_and_links():
    
    msm_links_dfs = {}
    counter = 0 
    temp_link_dfs = []
    temp_node_dfs = []

    for period in PERIOD_MAP:
        period_code = PERIOD_MAP[period]
        for scenario in BASE_SCENARIO_MAP:
            scen_code = SCENARIO_MAP[scenario]
            if scenario in ['2031', '2041', '2051']:
                model_code = str(year(scen_code)) + str(period_code) + "16"
                search_location = FUTURE_SHP_DIR + f'/Scenario_{model_code}/*.shp'
            else:
                model_code = str(year(scen_code)) + str(period_code) + str(scen_code)
                search_location = SHAPEFILE_DIR + '/*/*/*.shp'

            for file in glob.glob(search_location):
                if model_code in file:
                    if file.endswith('emme_links.shp'):
                        temp_df = gpd.read_file(file)[ROAD_LINKS_BASE_COLS]
                        temp_df['year'] = scenario
                        temp_link_dfs.append(temp_df)
                    elif file.endswith('emme_nodes.shp'):
                        # do once only
                        temp_node_dfs.append(gpd.read_file(file))
                        counter += 1

    msm_links_df = pd.concat(temp_link_dfs).drop_duplicates(subset=['ID', 'INODE', 'JNODE'])
    msm_nodes_df = pd.concat(temp_node_dfs).drop_duplicates(subset=['ID'])
    
    return msm_links_df, msm_nodes_df

msm_links_df, msm_nodes_df = read_nodes_and_links()

In [5]:
pnr_zones = msm_nodes_df[msm_nodes_df['ID']>90000]
pnr_zones.sjoin(MSM_ZONES, predicate='within')[['ID', 'MSM2018', 'LBA_Name', 'Sector_1', 'Sector_2', 'Sector_3']].to_csv(os.path.join(INPUT_DIR, 'PnR_Zones.csv'))

In [6]:
def map_nodes_to_zone(msm_nodes_df):
    return msm_nodes_df.sjoin_nearest(MSM_ZONES, how='left')[['ID', 'MSM2018', 'Sector_3']].rename(columns={'ID': 'NODE_ID'})
    # return msm_nodes_df.sjoin(MSM_ZONES, how='left', predicate='within')[['ID', 'MSM2018', 'Sector_3']].rename(columns={'ID': 'NODE_ID'})

def map_links(msm_links_df, node_zone_map):
    mapped_links_df = msm_links_df.copy().merge(node_zone_map, left_on='JNODE', right_on='NODE_ID')
    return mapped_links_df

def filter_road_links(msm_links_df):
    temp_df = msm_links_df[msm_links_df['TYPE'] > 0]
    msm_road_links_df = temp_df.merge(LINK_TYPE_MAP, left_on='TYPE', right_on='Link_Type', how='left')

    return msm_road_links_df

node_zone_map = map_nodes_to_zone(msm_nodes_df)
# msm_road_links_df = filter_road_links(map_links(msm_links_df, node_zone_map))

In [7]:
ROAD_LINKS_VOL_COLS =  ['ID', 'TYPE', 'MODES', 'LANES', 'VOLAX', 'VOLAU', 'VOLAD', 'TIMAU', '@vcv', '@hcv7', '@toll', '@veh', '@cars']

In [8]:
def rename_columns(suffix, original_cols = ROAD_LINKS_VOL_COLS, exceptions = ["ID"]):
    new_cols = {}
    for col in original_cols:
        if col not in exceptions:
            new_cols[col] = col + "_" + suffix
        else: 
            new_cols[col] = col 
    return new_cols

def get_road_volume_time_by_periods(links_dfs):
    link_vol_dfs = {}
    for period in PERIOD_MAP:
        period_code = PERIOD_MAP[period]
        base_df = links_dfs[period].copy()
        for scenario in SCENARIO_MAP:
            scen_code = SCENARIO_MAP[scenario]
            model_code = str(year(scen_code)) + str(period_code) + str(scen_code)
            print("Merging for model scenario: " + model_code)
            for file in glob.glob(SHAPEFILE_DIR + '/*/*/*.shp'):
                if model_code in file:
                    if file.endswith('emme_links.shp'):
                        temp_df = gpd.read_file(file)[ROAD_LINKS_VOL_COLS].rename(columns=rename_columns(model_code, ROAD_LINKS_VOL_COLS))
                        base_df = base_df.merge(temp_df, on='ID', how='left')
                        
        link_vol_dfs[period] = base_df
        del base_df

    return link_vol_dfs

def get_road_volume_time(links_df):
    
    for p, period in enumerate(PERIOD_MAP):
        period_code = PERIOD_MAP[period]
        for s, scenario in enumerate(SCENARIO_MAP):
            scen_code = SCENARIO_MAP[scenario]
            if scenario in ['2031', '2041', '2051']:
                model_code = str(year(scen_code)) + str(period_code) + "16"
                search_location = FUTURE_SHP_DIR + f'/Scenario_{model_code}/*.shp'
            else:
                model_code = str(year(scen_code)) + str(period_code) + str(scen_code)
                search_location = SHAPEFILE_DIR + '/*/*/*.shp'

            print("Merging for model scenario: " + model_code)
            for file in glob.glob(search_location):
                if model_code in file:
                    if file.endswith('emme_links.shp'):
                        if (p ==0) and (s==0):
                            base_df = links_df.copy()
                        temp_df = gpd.read_file(file)[ROAD_LINKS_VOL_COLS].rename(columns=rename_columns(model_code, ROAD_LINKS_VOL_COLS))
                        base_df = base_df.merge(temp_df, on='ID', how='left')
                        

    return base_df

msm_link_vols_df = get_road_volume_time(msm_links_df)
msm_link_vols_df.columns = msm_link_vols_df.columns.str.replace("@", "")

Merging for model scenario: 18112
Merging for model scenario: 26116
Merging for model scenario: 31116
Merging for model scenario: 41116
Merging for model scenario: 51116
Merging for model scenario: 18212
Merging for model scenario: 26216
Merging for model scenario: 31216
Merging for model scenario: 41216
Merging for model scenario: 51216
Merging for model scenario: 18312
Merging for model scenario: 26316
Merging for model scenario: 31316
Merging for model scenario: 41316
Merging for model scenario: 51316


In [9]:
msm_link_vols_df

,ID,INODE,JNODE,LENGTH,geometry,year,TYPE_18112,MODES_18112,LANES_18112,VOLAX_18112,...,LANES_51316,VOLAX_51316,VOLAU_51316,VOLAD_51316,TIMAU_51316,vcv_51316,hcv7_51316,toll_51316,veh_51316,cars_51316
0,1-10434,1,10434,8.947524,"LINESTRING (1744722.512 5988249.515, 1735792 5...",2018,1.0,abw,1.0,0.0,...,1.0,0.000000,193.43109,0.0,21.498823,0.880788,22.451939,0.0,0.0,170.97914
1,1-10448,1,10448,5.237727,"LINESTRING (1744722.512 5988249.515, 1747206 5...",2018,1.0,abw,1.0,0.0,...,1.0,0.000000,245.92143,0.0,20.210358,1.119802,21.281679,0.0,0.0,224.63976
2,2-10425,2,10425,0.674503,"LINESTRING (1736211.418 5982441.177, 1736501 5...",2018,1.0,abw,1.0,0.0,...,1.0,0.000000,187.56950,0.0,1.575295,0.863898,2.207146,0.0,0.0,185.36237
3,2-10427,2,10427,0.857741,"LINESTRING (1736211.418 5982441.177, 1736562 5...",2018,1.0,abw,1.0,0.0,...,1.0,13.489659,246.81114,0.0,3.434972,1.136750,15.103152,0.0,0.0,231.70799
4,3-10427,3,10427,2.108739,"LINESTRING (1737358.501 5985176.528, 1736562 5...",2018,1.0,abw,1.0,0.0,...,1.0,0.000000,148.92635,0.0,3.361699,0.500566,5.875201,0.0,0.0,143.05115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21161,13704-13705,13704,13705,0.012207,"LINESTRING (1761333 5912155, 1761326 5912145)",2051,NaN,NaN,NaN,NaN,...,1.0,0.000000,0.00000,0.0,-1.000000,0.000000,0.000000,0.0,0.0,0.00000
21162,13705-13704,13705,13704,0.012207,"LINESTRING (1761326 5912145, 1761333 5912155)",2051,NaN,NaN,NaN,NaN,...,1.0,0.000000,0.00000,0.0,-1.000000,0.000000,0.000000,0.0,0.0,0.00000
21163,13705-13706,13705,13706,0.013153,"LINESTRING (1761326 5912145, 1761313 5912143)",2051,NaN,NaN,NaN,NaN,...,1.0,0.000000,0.00000,0.0,-1.000000,0.000000,0.000000,0.0,0.0,0.00000
21164,13706-13587,13706,13587,1.362563,"LINESTRING (1761313 5912143, 1759956 5912020)",2051,NaN,NaN,NaN,NaN,...,1.0,0.000000,0.00000,0.0,-1.000000,0.000000,0.000000,0.0,0.0,0.00000


In [10]:
OUTPUT_DIR = '/Users/tszchun.chow/Library/CloudStorage/OneDrive-Arup/Auckland ToU Charging/10 Strategic Case/Graphics/data/Future Year Networks'
msm_link_vols_df.to_file(f'{OUTPUT_DIR}/FutureYearRoadNetwork.shp')

/var/folders/4x/lf34fbnd6sjcsfz9532hs7yrlpdrcj/T/ipykernel_36662/1337936984.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  msm_link_vols_df.to_file(f'{OUTPUT_DIR}/FutureYearRoadNetwork.shp')
/Users/tszchun.chow/Library/Caches/pypoetry/virtualenvs/akl-touch-KROpMQkL-py3.12/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'MODES_18112' to 'MODES_1811'
  ogr_write(
/Users/tszchun.chow/Library/Caches/pypoetry/virtualenvs/akl-touch-KROpMQkL-py3.12/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'LANES_18112' to 'LANES_1811'
  ogr_write(
/Users/tszchun.chow/Library/Caches/pypoetry/virtualenvs/akl-touch-KROpMQkL-py3.12/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'VOLAX_18112' to 'VOLAX_1811'
  ogr_write(
/Users/tszchun.chow/Library/Caches/pypoetry/virtualenvs/akl-touch-KROpMQkL-py3.